# Azure AI-agenter med stöd för Model Context Protocol (MCP)

Den här notebooken visar hur man använder Azure AI-agenter med verktyg för Model Context Protocol (MCP) för att skapa en intelligent agent som kan utnyttja externa MCP-servrar för förbättrade funktioner.


## Installera nödvändiga NuGet-paket

Först behöver vi installera Azure AI Agents Persistent-paketet som tillhandahåller kärnfunktionaliteten för att arbeta med Azure AI Agents.


## Fördelar med nyckellös autentisering

Den här anteckningsboken demonstrerar **nyckellös autentisering** som erbjuder flera fördelar:
- ✅ **Inga API-nycklar att hantera** - Använder Azure-baserad identitetsautentisering
- ✅ **Förbättrad säkerhet** - Inga hemligheter lagras i kod eller konfiguration
- ✅ **Automatisk uppdatering av autentiseringsuppgifter** - Azure hanterar autentiseringsuppgifternas livscykel
- ✅ **Rollbaserad åtkomst** - Använder Azure RBAC för detaljerade behörigheter

`DefaultAzureCredential` kommer automatiskt att använda den bästa tillgängliga autentiseringskällan:
1. Hanterad identitet (när den körs i Azure)
2. Azure CLI-autentiseringsuppgifter (under utveckling)
3. Visual Studio-autentiseringsuppgifter
4. Miljövariabler (om de är konfigurerade)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installera Azure Identity-paketet för autentisering med Azure-tjänster med DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importera nödvändiga namnrymder

Importera de nödvändiga namnrymderna för Azure AI Agents och Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurera Azure AI Agent Client (Nyckelfri autentisering)

Ställ in konfigurationsvariablerna och skapa PersistentAgentsClient med **nyckelfri autentisering**:
- **projectEndpoint**: Azure AI Foundry-projektets slutpunkt
- **modelDeploymentName**: Namnet på den distribuerade AI-modellen (GPT-4.1 nano)
- **mcpServerUrl**: URL till MCP-servern (Microsoft Learn API)
- **mcpServerLabel**: En etikett för att identifiera MCP-servern
- **DefaultAzureCredential**: Använder hanterad identitet, Azure CLI eller andra autentiseringskällor (inga API-nycklar krävs)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Skapa MCP-verktygsdefinition

Skapa en MCP-verktygsdefinition som ansluter till Microsoft Learn MCP-servern. Detta gör det möjligt för agenten att komma åt innehåll och dokumentation från Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Skapa AI-agenten

Skapa en ihållande AI-agent med den angivna modellen och MCP-verktygen. Agenten är konfigurerad med:
- GPT-4.1 nano-modellen
- Instruktioner att använda MCP-verktyg för assistans
- Tillgång till Microsoft Learn MCP-servern


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Skapa tråd och skicka meddelande

Skapa en konversationstråd och skicka ett användarmeddelande som frågar om skillnaden mellan Azure OpenAI och OpenAI. Detta kommer att testa agentens förmåga att använda MCP-verktygen för att ge korrekt information.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurera MCP-verktygsresurser (Nyckellös)

Ställ in MCP-verktygsresurserna. För en helt nyckellös metod kan du ta bort anpassade headers om MCP-servern stöder Azure-identitetsbaserad autentisering. Exemplet nedan visar hur man lägger till headers om det behövs, men för nyckellösa scenarier kan dessa vara överflödiga.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Starta agentkörning

Skapa och starta en körning för att bearbeta användarens meddelande. Agenten kommer att använda de konfigurerade MCP-verktygen och resurserna för att generera ett svar.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Övervaka körning och hantera verktygsgodkännanden (Keyless)

Övervaka agentens körstatus och hantera eventuella nödvändiga verktygsgodkännanden. Denna loop:
1. Väntar på att körningen ska slutföras eller kräva åtgärd
2. Godkänner automatiskt MCP-verktygsanrop när det behövs
3. För keyless-autentisering kan headers vara onödiga om MCP-servern stöder Azure-identitet


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Visa konversationsresultat

Hämta och visa alla meddelanden i tråden, inklusive både användarens fråga och agentens svar. Meddelandena visas i kronologisk ordning med tidsstämplar och rollindikatorer.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
